Applications of mOWL for Predicting Gene-Disease Associations

**mOWL Input dataset:** ontology in OWL format

**Methods input:** takes vectors of genes and diseases

**Output:** predict gene-disease association using cosine similarity


In [1]:
#Install the tool using:
! pip install gensim==4.0.0
! pip install mowl-borg

  error: subprocess-exited-with-error
  
  Preparing metadata (pyproject.toml) did not run successfully.
  exit code: 1
  
  [55 lines of output]
  running dist_info
  creating C:\Users\kkastampolidou\AppData\Local\Temp\pip-modern-metadata-vlymhh3n\gensim.egg-info
  writing C:\Users\kkastampolidou\AppData\Local\Temp\pip-modern-metadata-vlymhh3n\gensim.egg-info\PKG-INFO
  writing dependency_links to C:\Users\kkastampolidou\AppData\Local\Temp\pip-modern-metadata-vlymhh3n\gensim.egg-info\dependency_links.txt
  writing requirements to C:\Users\kkastampolidou\AppData\Local\Temp\pip-modern-metadata-vlymhh3n\gensim.egg-info\requires.txt
  writing top-level names to C:\Users\kkastampolidou\AppData\Local\Temp\pip-modern-metadata-vlymhh3n\gensim.egg-info\top_level.txt
  writing manifest file 'C:\Users\kkastampolidou\AppData\Local\Temp\pip-modern-metadata-vlymhh3n\gensim.egg-info\SOURCES.txt'
  Traceback (most recent call last):
    File "C:\Users\kkastampolidou\AppData\Local\Programs\Python\Pyth


     ---------------------------------------- 0.0/23.1 MB ? eta -:--:--
     ---------------------------------------- 0.1/23.1 MB 1.7 MB/s eta 0:00:14
     ---------------------------------------- 0.2/23.1 MB 2.8 MB/s eta 0:00:09
      --------------------------------------- 0.6/23.1 MB 4.5 MB/s eta 0:00:06
     - -------------------------------------- 0.9/23.1 MB 5.2 MB/s eta 0:00:05
     --- ------------------------------------ 1.8/23.1 MB 8.1 MB/s eta 0:00:03
     ---- ----------------------------------- 2.8/23.1 MB 10.6 MB/s eta 0:00:02
     ------ --------------------------------- 3.8/23.1 MB 12.0 MB/s eta 0:00:02
     ------ --------------------------------- 3.8/23.1 MB 12.0 MB/s eta 0:00:02
     -------- ------------------------------- 4.7/23.1 MB 11.6 MB/s eta 0:00:02
     --------- ------------------------------ 5.2/23.1 MB 11.6 MB/s eta 0:00:02
     ---------- ----------------------------- 6.2/23.1 MB 12.4 MB/s eta 0:00:02
     ------------ --------------------------- 7.1/23


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from google.colab import drive
drive.mount('/content/drive/')
#cd /content/drive/MyDrive/mowl-tutorial/
import os
os.chdir("/content/drive/MyDrive/Colab_Notebooks/mOWL2/data")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
# built-in imports
import sys
# sys.path.append('../../')
from sklearn.neural_network import MLPClassifier
from sklearn.metrics.pairwise import cosine_similarity
import json
import mowl
mowl.init_jvm("10g") #it was 4g
import torch as th
import logging
import numpy as np
import random
import math
import pickle as pkl
from mowl.visualization.base import TSNE
import matplotlib.pyplot as plt
from mowl.projection.edge import Edge
from mowl.projection import DL2VecProjector
from mowl.datasets import PathDataset
from mowl.datasets.builtin import GDADataset, GDAHumanDataset, GDAMouseDataset
from pykeen.models import TransE,ConvE,DistMult,TransR,TransD
from mowl.projection.dl2vec.model import DL2VecProjector
from mowl.kge import KGEModel
from mowl.evaluation.rank_based import EmbeddingsRankBasedEvaluator
from mowl.evaluation.base import TranslationalScore, CosineSimilarity
from mowl.projection.factory import projector_factory, PARSING_METHODS
from mowl.walking import DeepWalk
from gensim.models.word2vec import LineSentence
import gensim
from gensim.models import Word2Vec
from mowl.evaluation.rank_based import EmbeddingsRankBasedEvaluator
from mowl.evaluation.base import CosineSimilarity
from mowl.projection import TaxonomyWithRelationsProjector
from mowl.projection.edge import Edge

import warnings
warnings.filterwarnings(action='ignore',category=UserWarning,module='gensim')
warnings.filterwarnings(action='ignore',category=FutureWarning,module='gensim')

# Dataset

Build your own dataset

In [8]:
from mowl.ontology.extend import insert_annotations
'''
Parameters
- annotations (List of (str, str, bool) corresponding to (annotation file path, relation name, directed or undirected graph)) – Annotations to be included in the ontology. There can be more than one annotation file.
--> bidirectional (bool, optional) – If True, the triples will be considered undirected. Defaults to False
'''
# Prepare the annotations
diseases_annotations = ("data/diseases_annot.tsv", "http://has_annotation", True)
genes_annotations = ("data/genes_annot.tsv", "http://has_annotation", True)
gene_disease_associations = ("data/gene_disease_associations.tsv","http://is_associated_with/", True)

annotations = [diseases_annotations , genes_annotations, gene_disease_associations] # There could be more than 1 annotations file.

'''
Parameters
- ontology_file (str) – Ontology file in .owl format
- annotations (List of (str, str, bool) corresponding to (annotation file path, relation name, directed or undirected graph)) – Annotations to be included in the ontology. There can be more than one annotation file.
- out_file (str, optional) – Path for the new ontology. Defaults to ontology_file
- verbose (bool) – If true, information is shown.”
'''


# A new ontology created with the annotation
insert_annotations("data/upheno.owl", annotations, out_file = "data/upheno_with_annotations.owl" )


java.lang.IllegalArgumentException: java.lang.IllegalArgumentException: Illegal character in opaque part at index 7: file:C:\Users\kkastampolidou\Downloads\mOWL2\data\upheno_with_annotations.owl

Use the Built-in Dataset

In [14]:
dataset = GDAMouseDataset()

The dataset will be downloaded to a folder name `gda_mouse` with the training, validation and testing ontology dataset

In [15]:
! ls gda_mouse/

'ls' is not recognized as an internal or external command,
operable program or batch file.


# Graph-based embeddings

 ### Example for two methods: DL2vec and Owl2vec* methods

<font size="4">1) DL2vec Prediction Method </font>
This work is a graph-based machine-learning method to learn from biomedical ontologies. This method works by transforming the ontology into a graph following a set of rules. Random walks are generated from the obtained graph and then processed by a Word2Vec model, which generates embeddings of the original ontology classes. This algorithm is applied to generate numerical representations of genes and diseases based on the background knowledge found in the Gene Ontology, which was extended to incorporate phenotypes, functions of the gene products and anatomical location of gene expression. The representations of genes and diseases are then used to predict candidate genes for a given disease.

To show an example of DL2Vec, we need 3 components:

*   The ontology projector
*   The random walks generator
*   The Word2Vec model

1. **Projecting the ontology**
- Project the ontology using the DL2Vec Projector class, with the specific rules used to project the ontology.
- The outcome of the projection algorithm is an edgelist.



In [16]:
from mowl.projection.dl2vec.model import DL2VecProjector
projector = DL2VecProjector(True)
train_edges = projector.project(dataset.ontology)
test_edges = projector.project(dataset.testing)

2. **Generating random walks**
- The random walks are generated using the DeepWalk.


In [17]:
walker = DeepWalk(10, # number of walks per node
                  10, # walk length
                  0.1, # restart probability
                  workers=4, outfile = 'walk',seed=40) # number of threads

walks = walker.walk(train_edges)
walks_file = walker.outfile
sentences = LineSentence(walks_file)

3. **Training the Word2Vec model**
- To train the Word2Vec model, we rely on the Gensim library

In [18]:
model = Word2Vec(sentences, vector_size=100, epochs = 5, window=5, min_count=1, workers=10)


INFO:gensim.models.word2vec:collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #10000, processed 189362 words, keeping 29172 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #20000, processed 379216 words, keeping 48486 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #30000, processed 568826 words, keeping 63286 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #40000, processed 758168 words, keeping 75464 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #50000, processed 947694 words, keeping 85359 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #60000, processed 1137266 words, keeping 93885 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #70000, processed 1326756 words, keeping 101265 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #80000, processed 1516310 words, keepi

4. **Evaluating the embeddings**
- We are going to evaluate the plausibility of an association gene-disease with a gene against all possible diseases and check the rank of the true disease association using CosineSimilarity.


Evaluate the plausibility of gene-disease associations using cosine similarity.
Calculating cosine similarity to explore neighbors of each protein and finding most similar protein vectors. The interaction prediction is then performed based on similarity value based on the assumption that proteins with highly similar feature vectors are more like to interact

The gene-disease associations will be scoredc using cosine similarity. For that reason we use
the ``CosineSimilarity`` class.

In [19]:
genes, diseases = dataset.evaluation_classes
projector = TaxonomyWithRelationsProjector(taxonomy=False,
                                           relations=["http://is_associated_with"])

vectors = model.wv
evaluator = EmbeddingsRankBasedEvaluator(
    vectors,
    test_edges,
    CosineSimilarity,
    training_set=train_edges,
    head_entities = genes.as_str,
    tail_entities = diseases.as_str,
    device = 'cpu')


evaluator.evaluate(show=True)

INFO:root:Entity http://100126824 not present in the embeddings dictionary. Ignoring it.
INFO:root:Entity http://100503572 not present in the embeddings dictionary. Ignoring it.
INFO:root:Entity http://102294 not present in the embeddings dictionary. Ignoring it.
INFO:root:Entity http://104237 not present in the embeddings dictionary. Ignoring it.
INFO:root:Entity http://104245 not present in the embeddings dictionary. Ignoring it.
INFO:root:Entity http://104374 not present in the embeddings dictionary. Ignoring it.
INFO:root:Entity http://105193 not present in the embeddings dictionary. Ignoring it.
INFO:root:Entity http://107693 not present in the embeddings dictionary. Ignoring it.
INFO:root:Entity http://107986 not present in the embeddings dictionary. Ignoring it.
INFO:root:Entity http://108071 not present in the embeddings dictionary. Ignoring it.
INFO:root:Entity http://108072 not present in the embeddings dictionary. Ignoring it.
INFO:root:Entity http://108101 not present in th

Hits@1:   0.00 Filtered:   0.00
Hits@10:  0.12 Filtered:   0.17
Hits@100: 0.46 Filtered:   0.46
MR:       1171.59 Filtered: 1168.08
AUC:      0.86 Filtered:   0.86
Evaluation finished. Access the results using the "metrics" attribute.


In [20]:
print(evaluator.metrics)

{'hits@1': 0.004166666666666667, 'fhits@1': 0.004166666666666667, 'hits@10': 0.12083333333333333, 'fhits@10': 0.16666666666666666, 'hits@100': 0.4583333333333333, 'fhits@100': 0.4625, 'mean_rank': 1171.59375, 'fmean_rank': 1168.08125, 'rank_auc': 0.8620770405523719, 'frank_auc': 0.8625029847182426}


### Collect all the scores:

In [21]:
human_disease=[]
mouse_genes=[]
for classes in vectors.index_to_key:
    if 'OMIM' in classes:
        human_disease.append(classes)
    if classes[7:].isnumeric():
        mouse_genes.append(classes)

print(f'Number of the disease is {len(human_disease)}, and number of genes is {len(mouse_genes)}')

Number of the disease is 8383, and number of genes is 14048


In [22]:
human_disease[:10]

['http://OMIM_181500',
 'http://OMIM_601371',
 'http://OMIM_601665',
 'http://OMIM_222100',
 'http://OMIM_114480',
 'http://OMIM_608036',
 'http://OMIM_614286',
 'http://OMIM_601407',
 'http://OMIM_166710',
 'http://OMIM_601283']

In [23]:
mouse_genes[:10]

['http://11540',
 'http://17926',
 'http://111507',
 'http://16653',
 'http://11789',
 'http://13717',
 'http://14816',
 'http://56318',
 'http://12387',
 'http://19211']

In [24]:
human_disease_vectors=[]
for k in human_disease:
    human_disease_vectors.append(vectors[k])

mouse_genes_vectors=[]
for k in mouse_genes:
    mouse_genes_vectors.append(vectors[k])

from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(np.array(human_disease_vectors),np.array(mouse_genes_vectors))

print(f"The dimentions of this matrix is {similarity.shape}")


The dimentions of this matrix is (8383, 14048)


## Evaluating the predictions to find the most similar genes:

Finding Similar Genes for a Specific Disease: After evaluating the embeddings, you can find the top similar genes for a specific disease by using the find_similar_genes function.

In [25]:
def find_similar_genes(disease_id, top_k, disease_genes_similarity_matrix, disease_keys, gene_keys):
    disease_index = disease_keys.index(disease_id)
    prediction_list = np.flip(np.argsort(disease_genes_similarity_matrix[disease_index]))
    top_genes = [gene_keys[prediction_list[x]] for x in range(top_k)]

    return top_genes

#associations from the file MGI_DO.rpt


#DOID:0080449	developmental and epileptic encephalopathy 16	OMIM:615338	mouse, laboratory	10090	Tbc1d24	224617	MGI:2443456
disease_id = 'http://OMIM_615338'
top_k = find_similar_genes(disease_id, 10 ,similarity, human_disease, mouse_genes )
print(f'The most similar gene to disease {disease_id.split("/")[2]} are: {top_k}')


#DOID:0080436	developmental and epileptic encephalopathy 4	OMIM:612164	human	9606	STXBP1	6812
#DOID:0060309	syndromic X-linked intellectual disability		human	9606	HNRNPH2	3188
#HNRNPH2	3188	Hnrnph2	MGI:1201779	MP:0001186, MP:0005386, MP:0010771
#disease_id = 'http://OMIM_612164'
#top_k = find_similar_genes(disease_id, 5 ,similarity, human_disease, mouse_genes )
#print(f'The most similar gene to disease {disease_id.split("/")[2]} are: {top_k}')


#OMIM_181500 : schizophrenia : DOID:5419	OMIM:181500	mouse, laboratory	10090	Magi2	50791	MGI:1354953
#disease_id = 'http://OMIM_181500'
#top_k = find_similar_genes(disease_id, 5 ,similarity, human_disease, mouse_genes )
#print(f'The most similar gene to disease {disease_id.split("/")[2]} are: {top_k}')


#OMIM_615643 : neurodegeneration with brain iron accumulation 6
#DOID:0110740	OMIM:615643	mouse, laboratory	10090	Coasy	71743	MGI:1918993
#disease_id = 'http://OMIM_615643'
#top_k = find_similar_genes(disease_id, 5 ,similarity, human_disease, mouse_genes )
#print(f'The most similar gene to disease {disease_id.split("/")[2]} are: {top_k}')


The most similar gene to disease OMIM_615338 are: ['http://224617', 'http://16536', 'http://22346', 'http://93759', 'http://98985', 'http://20190', 'http://15394', 'http://11486', 'http://211286', 'http://14420']


In [26]:
def find_similar_genes(disease_id, top_k, disease_genes_similarity_matrix, disease_keys, gene_keys):
    disease_index = disease_keys.index(disease_id)
    prediction_list = np.flip(np.argsort(disease_genes_similarity_matrix[disease_index]))
    top_genes = [gene_keys[prediction_list[x]] for x in range(top_k)]

    return top_genes

disease_id = 'http://OMIM_615643'
top_k = find_similar_genes(disease_id, 5 ,similarity, human_disease, mouse_genes )
print(f'The most similar gene to disease {disease_id.split("/")[2]} are: {top_k}')


The most similar gene to disease OMIM_615643 are: ['http://14420', 'http://68816', 'http://71743', 'http://14069', 'http://20238']


<b> Tip:</b> You can use [NCBI](https://www.ncbi.nlm.nih.gov/gene/) to look for the Gene ID, or [Mouse Genome Informatics (MGI)](https://www.informatics.jax.org/quicksearch/summary?queryType=keywords&query)
For example : [Gene: 71743](https://www.ncbi.nlm.nih.gov/gene/?term=71743) in NCBI, and [Gene: 71743 - MGI:1918993](https://www.informatics.jax.org/marker/MGI:1918993) in MGI database.

</div>


# **Task 1 :**

<div class="alert alert-block alert-success" , color ='grreen'>

<font size="4">
    Predict the <font color='SteelBlue'>top 10 similar genes</font> to
    diabetes mellitus disease OMIM ID: <font color='Tomato'>http://OMIM_608036</font>
    using <font color='red'>OWL2vec*</font> prediction method

</font>

</div>


<div class="alert alert-block alert-info">
<b>Tip:</b> Follow the <b>TODO</b> interactions to modify the script, and the rest should be the same you just need to run the cell to execute the code.
</div>


----

<font size="4">2) OWL2vec* Prediction Method </font>

This example corresponds to the paper [Predicting candidate genes from phenotypes, functions and anatomical site of expression](https://doi.org/10.1093/bioinformatics/btaa879).

This work is a graph-based machine-learning method to learn from biomedical ontologies. This method works by transforming the ontology into a graph following a set of rules. Random walks are generated from the obtained graph and then processed by a Word2Vec model, which generates
embeddings of the original ontology classes. This algorithm is applied to generate numerical representations of genes and diseases based on the background knowledge found in the Gene Ontology, which was extended to incorporate phenotypes, functions of the gene products and anatomical location of gene expression. The representations of genes and diseases are then used to predict candidate genes for a given disease.

To show an example of DL2Vec, we need 3 components:

- The ontology projector
- The random walks generator
- The Word2Vec model

1. **Projecting the ontology**
- Project the ontology using the OWL2Vec* Projector class, with the specific rules used to project the ontology.
- The outcome of the projection algorithm is an edgelist.


In [27]:
from mowl.projection import OWL2VecStarProjector #TODO: import the appropriate function (refer to https://mowl.readthedocs.io/en/latest/api/projection/index.html)
dataset = GDAMouseDataset()
projector = OWL2VecStarProjector(True)
train_edges = projector.project(dataset.ontology)
test_edges = projector.project(dataset.testing)

2. **Generating random walks**
- The random walks are generated using the DeepWalk.


In [30]:
walker = DeepWalk( 10,#TODO: add the number of walks per node
                   10,#TODO: add the walk length
                  workers=10, # number of threads
                  outfile = 'walk2', #TODO: add the name of the output file for the walks
                  seed=40) #fix the random seed

walks = walker.walk(train_edges)
walks_file = walker.outfile
sentences = LineSentence(walks_file)

3. **Training the Word2Vec model**
- To train the Word2Vec model, we rely on the Gensim library

In [29]:
model2 = Word2Vec(sentences,
                 vector_size=100 , #TODO: add the size of the vector
                 epochs =5 ,     #TODO: update the number of training epochs
                 window=5, min_count=1, workers=10)


INFO:gensim.models.word2vec:collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #10000, processed 189734 words, keeping 36152 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #20000, processed 379120 words, keeping 59345 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #30000, processed 569024 words, keeping 76942 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #40000, processed 758630 words, keeping 89840 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #50000, processed 948388 words, keeping 99855 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #60000, processed 1138068 words, keeping 108130 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #70000, processed 1327214 words, keeping 114854 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #80000, processed 1516864 words, keep

4. **Evaluating the embeddings**
- We are going to evaluate the plausibility of an association gene-disease with a gene against all possible diseases and check the rank of the true disease association using CosineSimilarity.


In [31]:
genes, diseases = dataset.evaluation_classes
projector = TaxonomyWithRelationsProjector(taxonomy=False,
                                           relations=["http://is_associated_with"])

vectors = model2.wv
evaluator = EmbeddingsRankBasedEvaluator(
    vectors,
    test_edges,
    CosineSimilarity,
    training_set=train_edges,
    head_entities = genes.as_str,
    tail_entities = diseases.as_str,
    device = 'cpu')

evaluator.evaluate(show=True)

INFO:root:Entity http://100126824 not present in the embeddings dictionary. Ignoring it.
INFO:root:Entity http://100503572 not present in the embeddings dictionary. Ignoring it.
INFO:root:Entity http://102294 not present in the embeddings dictionary. Ignoring it.
INFO:root:Entity http://104237 not present in the embeddings dictionary. Ignoring it.
INFO:root:Entity http://104245 not present in the embeddings dictionary. Ignoring it.
INFO:root:Entity http://104374 not present in the embeddings dictionary. Ignoring it.
INFO:root:Entity http://105193 not present in the embeddings dictionary. Ignoring it.
INFO:root:Entity http://107693 not present in the embeddings dictionary. Ignoring it.
INFO:root:Entity http://107986 not present in the embeddings dictionary. Ignoring it.
INFO:root:Entity http://108071 not present in the embeddings dictionary. Ignoring it.
INFO:root:Entity http://108072 not present in the embeddings dictionary. Ignoring it.
INFO:root:Entity http://108101 not present in th

Hits@1:   0.00 Filtered:   0.00
Hits@10:  0.11 Filtered:   0.16
Hits@100: 0.47 Filtered:   0.47
MR:       1253.78 Filtered: 1250.24
AUC:      0.85 Filtered:   0.85
Evaluation finished. Access the results using the "metrics" attribute.


In [32]:
print(evaluator.metrics)

{'hits@1': 0.004166666666666667, 'fhits@1': 0.004166666666666667, 'hits@10': 0.1125, 'fhits@10': 0.15833333333333333, 'hits@100': 0.4708333333333333, 'fhits@100': 0.4708333333333333, 'mean_rank': 1253.7791666666667, 'fmean_rank': 1250.2354166666667, 'rank_auc': 0.8524106574339383, 'frank_auc': 0.8528362285100286}


In [33]:
human_disease=[]
mouse_genes=[]
for classes in vectors.index_to_key:
    if 'OMIM' in classes:
        human_disease.append(classes)
    if classes[7:].isnumeric():
        mouse_genes.append(classes)

print(f'Number of the disease is {len(human_disease)}, and number of genes is {len(mouse_genes)}')

Number of the disease is 8383, and number of genes is 14048


In [34]:
human_disease_vectors=[]
for k in human_disease:
    human_disease_vectors.append(vectors[k])

mouse_genes_vectors=[]
for k in mouse_genes:
    mouse_genes_vectors.append(vectors[k])

from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(np.array(human_disease_vectors),np.array(mouse_genes_vectors))

print("the dimentions of this matrix is ", similarity.shape)


the dimentions of this matrix is  (8383, 14048)


## Evaluating the predictions to find the most similar genes:

In [35]:
def find_similar_genes(disease_id, top_k, disease_genes_similarity_matrix, disease_keys, gene_keys):
    disease_index = disease_keys.index(disease_id)
    prediction_list = np.flip(np.argsort(disease_genes_similarity_matrix[disease_index]))
    top_genes = [gene_keys[prediction_list[x]] for x in range(top_k)]

    return top_genes



disease_id = 'http://OMIM_608036' #TODO: write the disease OMIM ID

number_of_genes = 10 #TODO: number of genes to be ranked

top_k = find_similar_genes( disease_id, #TODO: disease OMIM ID
                            number_of_genes, #TODO: number of genes
                           similarity,
                           human_disease,
                           mouse_genes)

print(f'The top {number_of_genes} most similar gene to disease {disease_id.split("/")[2]} are:')

for idx, genes in enumerate(top_k):
    print(f" Gene in Rank {idx+1} is : {top_k[idx]}")


The top 10 most similar gene to disease OMIM_608036 are:
 Gene in Rank 1 is : http://14235
 Gene in Rank 2 is : http://20526
 Gene in Rank 3 is : http://16147
 Gene in Rank 4 is : http://18095
 Gene in Rank 5 is : http://104099
 Gene in Rank 6 is : http://243382
 Gene in Rank 7 is : http://319387
 Gene in Rank 8 is : http://18595
 Gene in Rank 9 is : http://111346
 Gene in Rank 10 is : http://21817


<div class="alert alert-block alert-info">


<b> Tip:</b> You can use [NCBI](https://www.ncbi.nlm.nih.gov/gene/) to look for the Gene ID, or [Mouse Genome Informatics (MGI)](https://www.informatics.jax.org/quicksearch/summary?queryType=keywords&query)
For example : [Gene: 71743](https://www.ncbi.nlm.nih.gov/gene/?term=71743) in NCBI, and [Gene: 71743 - MGI:1918993](https://www.informatics.jax.org/marker/MGI:1918993) in MGI database.

</div>


In [36]:
!pip3 install sklearn

  error: subprocess-exited-with-error
  
  Getting requirements to build wheel did not run successfully.
  exit code: 1
  
  [15 lines of output]
  The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
  rather than 'sklearn' for pip commands.
  
  Here is how to fix this error in the main use cases:
  - use 'pip install scikit-learn' rather than 'pip install sklearn'
  - replace 'sklearn' by 'scikit-learn' in your pip requirements files
    (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
  - if the 'sklearn' package is used by one of your dependencies,
    it would be great if you take some time to track which package uses
    'sklearn' instead of 'scikit-learn' and report it to their issue tracker
  - as a last resort, set the environment variable
    SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
  
  More information is available at
  https://github.com/scikit-learn/sklearn-pypi-package
  [end of output]
  
  note: This error originates f


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


----------------------------------------

# Syntactic embeddings

In [37]:
from mowl.corpus import extract_and_save_axiom_corpus
from mowl.owlapi import OWLAPIAdapter
from mowl.reasoning import MOWLReasoner
from org.semanticweb.elk.owlapi import ElkReasonerFactory
from java.util import HashSet
from gensim.models.word2vec import LineSentence
from gensim.models import Word2Vec
import os

<font size="4">1) Onto2Vec Prediction Method </font>


This example corresponds to the paper **Onto2Vec: joint vector-based representation of biological entities and their ontology-based annotations**.

This method is an approach to learn numerical representations (embeddings) of (biomedical) ontologies by representing ontology axioms as text sequences and applying an unsupervised learning algorithm such as Word2Vec. Onto2Vec uses an ontology reasoner to infer new axioms as a preprocessing step. The algorithm is tested on the protein-protein interaction task.

For this algorithm, we need three components:

1. The reasoner

2. The corpus generator

3. The Word2Vec model


**1) Inferring new axioms**

- Onto2Vec uses an ontology reasoner to infer new axioms as a preprocessing step. In the original paper, the authors used the HermiT reasoner. For this example, we use the ELK reasoner.


In [38]:
reasoner_factory = ElkReasonerFactory()
reasoner = reasoner_factory.createReasoner(dataset.ontology)
mowl_reasoner = MOWLReasoner(reasoner)


In [39]:
# %%
# We wrap the reasoner into the :class:`MOWLReasoner <mowl.reasoning.base.MOWLReasoner>` class \
# in order to use some shortcuts the mOWL
# provides such as:
#
# - inferring subclass axioms
# - inferring equivalent class axioms
# - inferring disjoint axioms (not applicable for this example since we use ELK reasoner)

classes = dataset.ontology.getClassesInSignature()
subclass_axioms = mowl_reasoner.infer_subclass_axioms(classes)
equivalent_class_axioms = mowl_reasoner.infer_equivalent_class_axioms(classes)

# %%
# We can now add the inferred axioms to the ontology.

adapter = OWLAPIAdapter()
manager = adapter.owl_manager

axioms = HashSet()
axioms.addAll(subclass_axioms)
axioms.addAll(equivalent_class_axioms)

manager.addAxioms(dataset.ontology, axioms)

INFO:root:Number of inferred axioms: 4384772.
INFO:root:Number of inferred axioms: 177316.


<java object 'org.semanticweb.owlapi.model.parameters.ChangeApplied'>

**2- The corpus generator**

In [40]:
extract_and_save_axiom_corpus(dataset.ontology, "onto2vec_corpus.txt")

INFO:root:Generating axioms corpus


UnicodeEncodeError: 'charmap' codec can't encode character '\xe9' in position 137: character maps to <undefined>

**3- Generating the corpus and training the model**
- Now that we have an extended ontology, we can generate the corpus out of it. After that, we can train the Word2Vec model.


In [42]:
sentences = LineSentence("data/onto2vec_corpus.txt")
model_onto = Word2Vec(sentences, vector_size=5, epochs=10, window=2, min_count=1, workers=4)

# %%
# Cleaning up memory
# os.remove("onto2vec_corpus.txt")

INFO:gensim.models.word2vec:collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #10000, processed 38468 words, keeping 12923 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #20000, processed 77131 words, keeping 23109 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #30000, processed 115725 words, keeping 32049 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #40000, processed 154454 words, keeping 40249 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #50000, processed 194065 words, keeping 47947 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #60000, processed 233808 words, keeping 55406 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #70000, processed 272657 words, keeping 62070 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #80000, processed 311496 words, keeping 683

## Evaluating the embeddings
- We are going to evaluate the plausibility of an association gene-disease with a gene against all possible diseases and check the rank of the true disease association using CosineSimilarity.


In [1]:
genes, diseases = dataset.evaluation_classes
projector = TaxonomyWithRelationsProjector(taxonomy=False,
                                           relations=["http://is_associated_with"])

vectors = model_onto.wv
evaluator = EmbeddingsRankBasedEvaluator(
    vectors,
    test_edges,
    CosineSimilarity,
    training_set=train_edges,
    head_entities = genes.as_str,
    tail_entities = diseases.as_str,
    device = 'cpu')

evaluator.evaluate(show=True)

NameError: name 'dataset' is not defined

In [ ]:
human_disease=[]
mouse_genes=[]
for classes in vectors.index_to_key:
    if 'OMIM' in classes:
        human_disease.append(classes)
    if classes[7:].isnumeric():
        mouse_genes.append(classes)

human_disease_vectors=[]
for k in human_disease:
    human_disease_vectors.append(vectors[k])

mouse_genes_vectors=[]
for k in mouse_genes:
    mouse_genes_vectors.append(vectors[k])

similarity = cosine_similarity(np.array(human_disease_vectors),np.array(mouse_genes_vectors))

disease_id = 'http://OMIM_612164'
top_k = find_similar_genes(disease_id, 10 ,similarity, human_disease, mouse_genes )
print(f'The most similar gene to disease {disease_id.split("/")[2]} are: {top_k}')


The most similar gene to disease OMIM_612164 are: ['http://18205', 'http://124773', 'http://50604', 'http://26092', 'http://20400', 'http://18383', 'http://51426', 'http://56993', 'http://387882', 'http://64744']


<div class="alert alert-block alert-info">


<b> Tip:</b> You can use [NCBI](https://www.ncbi.nlm.nih.gov/gene/) to look for the Gene ID, or [Mouse Genome Informatics (MGI)](https://www.informatics.jax.org/quicksearch/summary?queryType=keywords&query)
For example : [Gene: 71743](https://www.ncbi.nlm.nih.gov/gene/?term=71743) in NCBI, and [Gene: 71743 - MGI:1918993](https://www.informatics.jax.org/marker/MGI:1918993) in MGI database.

</div>

----

# **Task 2 :**

<div class="alert alert-block alert-success">

<font size="4">
    Predict the <font color='SteelBlue'>top 5 similar genes</font> to
    diabetes mellitus disease OMIM ID: <font color='Tomato'>http://OMIM_608036</font>
    using <font color='red'>OPA2Vec*</font> prediction method

</font>

</div>

<div class="alert alert-block alert-info">
<b>Tip:</b> Follow the <b>TODO</b> interactions to modify the script, and the rest should be the same you just need to run the cell to execute the code.
</div>


---

<font size="4">2) OPA2Vec Prediction Method </font>


This example corresponds to the paper **OPA2Vec: combining formal and informal content of biomedical ontologies to improve similarity-based prediction**.

This method is an extension of **Onto2Vec** that apart from formal knowldege (i.e. axioms) it also uses informal knowledge such as entity metadata (i.e. synonyms, definitions, etc.)

For this algorithm, we need four components:

1.  The reasoner

2. The corpus generator

<font color='red'>3. The annotations generator</font>

4. The Word2Vec model

In [ ]:
from mowl.corpus import extract_and_save_axiom_corpus
from mowl.owlapi import OWLAPIAdapter
from mowl.reasoning import MOWLReasoner
from org.semanticweb.elk.owlapi import ElkReasonerFactory
from java.util import HashSet
from gensim.models.word2vec import LineSentence
from gensim.models import Word2Vec
import os
from sklearn.metrics.pairwise import cosine_similarity

from mowl.corpus import extract_and_save_axiom_corpus, extract_and_save_annotation_corpus
# OPA2Vec use annotation so we need to import extract_and_save_annotation_corpus


**1) Inferring new axioms**

- OPA2Vec uses an ontology reasoner to infer new axioms as a preprocessing step. In the original paper, the authors used the HermiT reasoner. For this example, we use the ELK reasoner.



In [ ]:
reasoner_factory = ElkReasonerFactory()
reasoner = reasoner_factory.createReasoner(dataset.ontology)
mowl_reasoner = MOWLReasoner(reasoner)

We wrap the reasoner into the **MOWLReasoner** class in order to use some shortcuts the mOWL provides such as:

- inferring subclass axioms

- inferring equivalent class axioms

- inferring disjoint axioms (not applicable for this example since we use ELK reasoner)


In [ ]:
classes = dataset.ontology.getClassesInSignature()
subclass_axioms = mowl_reasoner.infer_subclass_axioms(classes)
equivalent_class_axioms = mowl_reasoner.infer_equivalent_class_axioms(classes)

We can now add the inferred axioms to the ontology:

**2- The corpus generator**

In [ ]:
adapter = OWLAPIAdapter()
manager = adapter.owl_manager

axioms = HashSet()
axioms.addAll(subclass_axioms)
axioms.addAll(equivalent_class_axioms)

manager.addAxioms(dataset.ontology, axioms)

In [ ]:
extract_and_save_axiom_corpus(dataset.ontology, "opa2vec_corpus.txt")

<font color='red'> **3- The annotations generator**</font>

In [ ]:
#TODO : extract and save the annotations (dataset.ontology, "opa2vec_corpus.txt", mode="a")


**4- Generating the corpus and training the model**
- Now that we have an extended ontology, we can generate the corpus out of it. After that, we can train the Word2Vec model.



In [ ]:
sentences = LineSentence("opa2vec_corpus.txt")

model = Word2Vec(sentences,
                 vector_size= , #TODO: add the size of the vector
                 epochs = ,     #TODO: update the number of training epochs
                 window=5, min_count=1, workers=10)


## Evaluating the embeddings

In [ ]:
genes, diseases = dataset.evaluation_classes
projector = TaxonomyWithRelationsProjector(taxonomy=False,
                                           relations=["http://is_associated_with"])

eval_train_edges = projector.project(dataset.ontology)
eval_test_edges = projector.project(dataset.testing)

vectors = model.wv
evaluator = EmbeddingsRankBasedEvaluator(
    vectors,
    eval_test_edges,
    CosineSimilarity,
    training_set=eval_train_edges,
    head_entities = genes.as_str,
    tail_entities = diseases.as_str,
    device = 'cpu')

evaluator.evaluate(show=True)

In [ ]:
def find_similar_genes(disease_id, top_k, disease_genes_similarity_matrix, disease_keys, gene_keys):
    disease_index = disease_keys.index(disease_id)
    prediction_list = np.flip(np.argsort(disease_genes_similarity_matrix[disease_index]))
    top_genes = [gene_keys[prediction_list[x]] for x in range(top_k)]

    return top_genes


vectors = model.wv


human_disease=[]
mouse_genes=[]
for classes in vectors.index_to_key:
    if 'OMIM' in classes:
        human_disease.append(classes)
    if 'http://' in classes and classes[7:].isnumeric():
        mouse_genes.append(classes)

human_disease_vectors=[]
for k in human_disease:
    human_disease_vectors.append(vectors[k])

mouse_genes_vectors=[]
for k in mouse_genes:
    mouse_genes_vectors.append(vectors[k])

similarity = cosine_similarity(np.array(human_disease_vectors),np.array(mouse_genes_vectors))


In [ ]:
disease_id =  #TODO: write the disease OMIM ID

number_of_genes = #TODO: number of genes to be ranked

top_k = find_similar_genes( , #TODO: disease OMIM ID
                            , #TODO: number of genes
                           similarity,
                           human_disease,
                           mouse_genes)

print(f'The top {number_of_genes} most similar gene to disease {disease_id.split("/")[2]} are:')

for idx, genes in enumerate(top_k):
    print(f" Gene in Rank {idx+1} is : {top_k[idx]}")


------

# Ontologies and Text-mining

Genes and Diseases extracted from text using the exact match, refer to the notebook [Ontologies_and_text_mining](https://github.com/bio-ontology-research-group/mowl-tutorial/blob/main/notebooks/02_Ontologies_and_text_mining.ipynb), to see how the model trained, and for more details refer to the presentation [Ontologies and Text-mining](https://github.com/bio-ontology-research-group/mowl-tutorial/blob/main/slides/Ontologies%20and%20text%20mining.pdf).


In [ ]:
from gensim.models import Word2Vec
import pickle as pkl
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [ ]:
# Collect the vectors for the genes and diseases
word2vec_file = 'w2v_model/wv_model'
vectors = Word2Vec.load(word2vec_file)

human_disease=[]
mouse_genes=[]
for classes in vectors.wv.index_to_key:
    if 'OMIM_' in classes:
        human_disease.append(classes)
    if 'http://' in classes and classes[7:].isnumeric():
        mouse_genes.append(classes)

human_disease_vectors=[]
for k in human_disease:
    human_disease_vectors.append(vectors.wv[k])

mouse_genes_vectors=[]
for k in mouse_genes:
    mouse_genes_vectors.append(vectors.wv[k])

similarity = cosine_similarity(np.array(human_disease_vectors),np.array(mouse_genes_vectors))


## Evaluating the predictions to find the most similar genes:

In [ ]:
# Evaluating the similarity

def find_similar_genes(disease_id, top_k, disease_genes_similarity_matrix, disease_keys, gene_keys):
    disease_index = disease_keys.index(disease_id)
    prediction_list = np.flip(np.argsort(disease_genes_similarity_matrix[disease_index]))
    top_genes = [gene_keys[prediction_list[x]] for x in range(top_k)]

    return top_genes

disease_id = 'http://OMIM_114500'
# DOID:9256	colorectal cancer	OMIM:114500	human	9606	KDR	3791


top_k = find_similar_genes(disease_id, 10 ,similarity, human_disease, mouse_genes )
print(f'The most similar genes to disease {disease_id.split("/")[2]} are:')
for i, j in enumerate(top_k):
    print(i+1, j)

# The linked genes at rank 7

The most similar genes to disease OMIM_114500 are:
1 http://4838
2 http://5045
3 http://572
4 http://4942
5 http://5805
6 http://5064
7 http://3791
8 http://6418
9 http://55806
10 http://22944




---

This code is a comprehensive example of using various methods for predicting gene-disease associations based on ontology and text mining. Let's break down the code into its main components and understand what each part does:

Imports and Setup: The code starts by importing necessary libraries and setting up the environment, including installing required packages, mounting Google Drive, and changing the working directory.

Dataset Preparation: It provides methods to build or use built-in datasets for gene-disease associations. These datasets are used for training and testing the prediction models.

Graph-Based Embeddings (DL2vec):

Projecting the Ontology: It projects the ontology into a graph format suitable for further processing.
Generating Random Walks: Random walks are generated from the graph structure.
Training the Word2Vec Model: Word2Vec model is trained on the generated random walks.
Evaluating the Embeddings: Cosine similarity is used to evaluate the plausibility of gene-disease associations.
Syntactic Embeddings (Onto2Vec, OPA2Vec): Similar to DL2vec, these methods involve projecting the ontology, generating random walks, training Word2Vec models, and evaluating embeddings. However, they may involve additional steps such as inferring new axioms and extracting annotations.

Ontologies and Text Mining: This section utilizes pre-trained Word2Vec embeddings to find similar genes for a given disease based on text mining.

Task Instructions: Instructions are provided for specific tasks, such as predicting similar genes for a given disease using different prediction methods.

Each method follows a similar pattern:

Preprocess the ontology or text data.
Train a Word2Vec model on the processed data.
Evaluate the trained embeddings using cosine similarity or other metrics.